In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [4]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [8]:
!pip install transformers
import pandas as pd
# Recommended tensorflow version is <= 2.1.0, otherwise F1 score function breaks
import tensorflow as tf
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential,Model
from keras.layers import Dense, Activation, Flatten,concatenate
import tensorflow_datasets as tfds
from transformers import TFRobertaForSequenceClassification
from transformers import RobertaTokenizer
import os
import re

In [9]:
train_def=pd.read_csv('dftrain')
datos_def=pd.read_csv('dfdatos')

In [13]:


def remTexto(x):
  x=x.casefold()
  x=re.sub(r'http\S*', 'http:',x)
  x=re.sub(r'[^a-z\s]', '',x)
  return x

# Load your Dataset
train_tweets = pd.read_csv('train.csv')
test_tweets = pd.read_csv('test.csv')

train_tweets['text']=train_tweets['text'].apply(lambda x:remTexto(x))
test_tweets['text']=test_tweets['text'].apply(lambda x:remTexto(x))

training_sentences, testing_sentences = train_test_split(train_tweets[['text', 'target']],
                                                         test_size=0.2)

roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# can be up to 512 for BERT
max_length = 150

# the recommended batches size for BERT are 32,64 ... however on this dataset we are overfitting quite fast
# and smaller batches work like a regularization.

batch_size = 32

def convert_example_to_feature(text):
    return roberta_tokenizer.encode_plus(text,
                                 add_special_tokens=True,  # add [CLS], [SEP]
                                 max_length=max_length,  # max length of the text that can go to RoBERTa
                                 pad_to_max_length=True,  # add [PAD] tokens at the end of sentence
                                 return_attention_mask=True,  # add attention mask to not focus on pad tokens
                                 truncation=True)

# map to the expected input to TFRobertaForSequenceClassification, see here
def map_example_to_dict(input_ids, attention_masks, label):
    return {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
           }, label

def encode_examples(ds, limit=-1):
    # Prepare Input list
    input_ids_list = []
    attention_mask_list = []
    label_list = []

    if (limit > 0):
        ds = ds.take(limit)

    for review, label in tfds.as_numpy(ds):
        bert_input = convert_example_to_feature(review.decode())
        input_ids_list.append(bert_input['input_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append([label])

    return tf.data.Dataset.from_tensor_slices((input_ids_list,
                                               attention_mask_list,
                                               label_list)).map(map_example_to_dict)

training_sentences_modified = tf.data.Dataset.from_tensor_slices((training_sentences['text'],
                                                                  training_sentences['target']))

testing_sentences_modified = tf.data.Dataset.from_tensor_slices((testing_sentences['text'],
                                                                 testing_sentences['target']))

ds_train_encoded = encode_examples(training_sentences_modified).shuffle(10000).batch(batch_size)
ds_test_encoded = encode_examples(testing_sentences_modified).batch(batch_size)



learning_rate = 7e-5
number_of_epochs = 5


# model initialization
model = TFRobertaForSequenceClassification.from_pretrained("roberta-base")

NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = train_def.shape[1], activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(1, kernel_initializer='normal',activation='sigmoid'))

# Compile the network :

model_concat = concatenate([model.output, NN_model.output], axis=-1)
model_concat = Dense(1, activation='softmax')(model_concat)
model2 = Model(inputs=[model.input, NN_model.input], outputs=model_concat)

model2.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam')

Some weights of the model checkpoint at roberta-base were not used when initializing TFRobertaForSequenceClassification: ['lm_head']
- This IS expected if you are initializing TFRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: ignored

In [ ]:
history=model.fit([ds_train_encoded,train_def], epochs=number_of_epochs,
          validation_data=ds_test_encoded)

In [ ]:
import matplotlib.pyplot as plt


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc = 'upper left')
plt.show()

In [ ]:
#Tengo que agregar esto por como esta definido tensor slices del dataset original
#Luego sera ignorado por el modelo
test_tweets['tar']=0
submission_sentences_modified = tf.data.Dataset.from_tensor_slices((test_tweets['text'],test_tweets['tar']))
test_encoded = encode_examples(submission_sentences_modified).batch(batch_size)
x=model.predict([test_encoded,test_def])


In [ ]:
submission_pre = tf.nn.softmax(x[0])
submission_pre_argmax = tf.math.argmax(submission_pre, axis=1)
submission_pre_argmax

In [ ]:
res=pd.DataFrame()
res['target']=submission_pre_argmax.numpy()
res['id']=test_tweets['id']

In [ ]:
res.to_csv('resultadosRoberta.csv',index=False)

In [ ]:
#!kaggle competitions submit -c nlp-getting-started -f resultadosRoberta.csv -m "Nuevo intento"